## Importing Packages

In [1]:
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import netCDF4
from netCDF4 import Dataset
import numpy  as np
import scipy

import cartopy.crs as ccrs
import cartopy
from cartopy.io.shapereader import Reader
from cartopy.feature import ShapelyFeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import matplotlib
import matplotlib.dates as mdates
from matplotlib.ticker import AutoMinorLocator, StrMethodFormatter
from matplotlib.pyplot import cm
from matplotlib.lines import Line2D

from gsw import p_from_z
from gsw import pot_rho_t_exact as prho

import seaborn as sns

import datetime

from scipy import interpolate, ndimage
from PIL import Image
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

import sys
import subprocess

from datetime import datetime, date

import glob
import os

import warnings
warnings.filterwarnings("ignore")

## Dataset

#### Glorys Dataset

In [2]:
# # Get a list of all .nc files available in different folders
filenames = glob.glob("d:/00_Masters/01_Dados/Mercator/*.nc")
dsmerged = xr.open_mfdataset(filenames)

# # Corrige erro de datas duplicadas pelo mfdataset
_, index = np.unique(dsmerged['time'], return_index=True)
dsmerged = dsmerged.isel(time=index)

### Select area and export 

In [ ]:
# ESTAÇÃO JP
JP = dsmerged.sel(latitude=-6.5, longitude=-34.75, method='nearest')
# JP['thetao'].to_dataframe()
JP['thetao'].to_netcdf('d:/00_Masters/01_Dados/Station_Cases/Glorys_Station_JP.nc')

In [3]:
# ESTAÇÃO PE - Ponta das Pedras / Ilha de Itamaracá
PE_2 = dsmerged.sel(latitude=-7.75, longitude=-34.50, method='nearest')
# PE_2['thetao'].to_dataframe()
PE_2['thetao'].to_netcdf('d:/00_Masters/01_Dados/Station_Cases/Glorys_Station_PE_2.nc')

In [4]:
# ESTAÇÃO PE - Cabo de Santo Agostinho
PE_CSA = dsmerged.sel(latitude=-8.33, longitude=-34.66, method='nearest')
# PE_CSA['thetao'].to_dataframe()
PE_CSA['thetao'].to_netcdf('d:/00_Masters/01_Dados/Station_Cases/Glorys_Station_PE_CSA.nc')

In [5]:
# ESTAÇÃO AL - Maragogigi
AL = dsmerged.sel(latitude=-9.17, longitude=-34.92, method='nearest')
# AL['thetao'].to_dataframe()
AL['thetao'].to_netcdf('d:/00_Masters/01_Dados/Station_Cases/Glorys_Station_AL.nc')

In [6]:
# ESTAÇÃO SERGIPE - Canyon Sao Francisco / Canyon do Japaratuba
SE = dsmerged.sel(latitude=-11.0, longitude=-36.75, method='nearest')
# SE['thetao'].to_dataframe()
SE['thetao'].to_netcdf('d:/00_Masters/01_Dados/Station_Cases/Glorys_Station_AL_CSF.nc')

In [ ]:
# ESTAÇÃO PRAIA DO FORTE - A0
PF = dsmerged.interp(latitude=-12.60288903576, longitude=-37.974646744616, method='quadratic')
# PF = PF['thetao'].sel(depth=29,method='nearest')
# PF = PF.to_dataframe()
PF['thetao'].to_netcdf('d:/00_Masters/01_Dados/Station_Cases/Glorys_Station_PF.nc')

In [ ]:
# ESTAÇÃO BAIA DE CAMAMU - Canyon Guaibim
BAcam = dsmerged.sel(latitude=-13.33, longitude=-38.75, method='nearest')
# BAcam['thetao'].to_dataframe()
BAcam['thetao'].to_netcdf('d:/00_Masters/01_Dados/Station_Cases/Glorys_Station_BA_CAM.nc')

-----

In [ ]:
# df_list = [PF,SE,BAcam,AL,PE_CSA,PE_2,JP]
df_list = PF

In [ ]:
cutoff_value = -.2

temp_list = []
eventos_list = []
dates_list = []

for i,n in enumerate(df_list.index):
    if i < (len(df_list)-1):
        e = df_list.thetao[i+1] - df_list.thetao[i]
        d = df_list.index[i+1]
        t = df_list.thetao[i+1]
        if (e <= cutoff_value):
            temp_list.append(t)
            eventos_list.append(e)
            dates_list.append(d)
        elif (len(dates_list)>1) and (dates_list[-1] == df_list.index[i]) and (e < 0):
            temp_list.append(t)
            eventos_list.append(e)
            dates_list.append(d)


eventos = pd.DataFrame({'time': dates_list,'temp':temp_list,'diff': eventos_list})
eventos.set_index('time',inplace=True,drop=True)
eventos.to_csv('D:/00_Masters/03_Figuras_Finais/Eventos_PF.csv',sep=';',decimal=',')


# Figuras

#### Perfil Temperatura

In [ ]:
fig, ax = plt.subplots()

depth = [0.494025,   5.078224,   9.572997,  15.81007 ,  18.49556 ,  21.59882 ,  25.21141 ,  29.44473, 34.43415 ]

ax.set_ylim(23,29)

color=cm.coolwarm_r(np.linspace(0,1,9))
for d,c in zip(depth,color):
    p = Merc_PF.sel(depth=d,method='nearest')
    p['thetao'].plot(c=c)


plt.legend(labels=['0.5m','5.0m','9.6m','15.8m','18.5m','21.6m','25.2m','29.4m','34.4m'],
           bbox_to_anchor=(1.01, 1), loc='upper left')
plt.grid(linestyle='--',alpha=0.7,which='both')
ax.set_xlabel('')
ax.set_ylabel('Temp. (°C)')

ax.set_title('')

ax.xaxis.set_major_locator(
    mdates.MonthLocator(bymonth=(11,12,1,2,3,4)))
ax.xaxis.set_minor_locator(
    mdates.DayLocator(bymonthday=(10,20)))

ax.xaxis.set_major_formatter(
    mdates.ConciseDateFormatter(ax.xaxis.get_major_locator()))

ax.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))
ax.xaxis.set_minor_formatter(mdates.DateFormatter('%d'))

ax.tick_params(which='minor', width=1, 
               labelsize=8.5)
ax.tick_params(which='major', length=10, 
               pad=4.2, labelrotation=0,)

ax.set_xlim(17471,17625)
# ax.set_xlim('2017-11-01','2018-04-05')

plt.savefig('D:/Users/julia/Desktop/Academia/01_Mestrado/Resultados/PF-Merc_depths.png', format='png', dpi=300, bbox_inches='tight')
plt.show()

##### PERFIL VERTICAL DE TEMPERATURA - COUNTOURF

In [ ]:
temp = Merc_PF['thetao'].sel(depth=slice(0,35))
t = temp.values
t = t.transpose()
ti, d = np.meshgrid(temp.time, temp.depth,sparse=False)

In [ ]:
fig, [ax,ax2] = plt.subplots(1, 2, sharey=True, gridspec_kw={'width_ratios': [3, 1]})
fig.set_figheight(2)
fig.set_figwidth(15)
fig.subplots_adjust(wspace=0.015)
cbar_ax = fig.add_axes([.91, 0.15, 0.01, 0.7])

vmin = 24
vmax = 28
levels = np.arange(24,28.2,.25)
norm = matplotlib.colors.Normalize(vmin=vmin, vmax=vmax)

# norm = matplotlib.colors.Normalize(vmin=24, vmax=28)
# c = ax.pcolormesh(ti, d, t,cmap='jet',vmin=24, vmax=28, shading='gouraud', snap=True)
# ax2.pcolormesh(ti, d, t,cmap='jet',vmin=24, vmax=28, shading='gouraud', snap=True)
# cbar = fig.colorbar(c, cax=cbar_ax,  ticks=[24,26,28], orientation="vertical",) #shrink=.8 sm
# cbar.set_label(label='Temp. (°C)', size=12)
# cbar.ax.tick_params(labelsize=12)

# Sem shading
c = ax.contourf(ti, d, t,cmap='jet',vmin=vmin, vmax=vmax, levels=levels, extend='both')
ax2.contourf(ti, d, t,cmap='jet',vmin=vmin, vmax=vmax, levels=levels, extend='both')
# sm = plt.cm.ScalarMappable(norm=norm, cmap = c.cmap)
# sm.set_array([])
cbar = fig.colorbar(c, cax=cbar_ax, ticks=[24,26,28], orientation="vertical",) #shrink=.8 sm
cbar.set_label(label='°C', size=12, rotation=0, labelpad=15,loc='top' )
cbar.ax.tick_params(labelsize=12)


# Zoom out
# ax.invert_yaxis()
ax.xaxis.set_major_locator(
    mdates.MonthLocator(bymonth=(11,12,1,2,3)))
ax.xaxis.set_minor_locator(
    mdates.DayLocator(bymonthday=(1,7,14,21,28)))
ax.xaxis.set_major_formatter(
    mdates.ConciseDateFormatter(ax.xaxis.get_major_locator()))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))
ax.xaxis.set_minor_formatter(mdates.DateFormatter('%d'))
ax.tick_params(which='minor', width=1.0, 
               labelsize=8.5)
ax.tick_params(which='major', length=10, 
               pad=4.2, labelrotation=0,)
# ax.set_xlim('2017-11-01','2018-04-05')
ax.set_xlim(17471-16,17591)


# zoom in
ax2.invert_yaxis()
ax2.xaxis.set_major_locator(
    mdates.MonthLocator(bymonth=(11,12,1,2,3)))
ax2.xaxis.set_minor_locator(
    mdates.DayLocator(bymonthday=(1,7,14,21,28)))
ax2.xaxis.set_major_formatter(
    mdates.ConciseDateFormatter(ax.xaxis.get_major_locator()))
ax2.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))
ax2.xaxis.set_minor_formatter(mdates.DateFormatter('%d'))
ax2.tick_params(which='minor', width=1.0, 
               labelsize=8.5)
ax2.tick_params(which='major', length=10, 
               pad=4.2, labelrotation=0,)
# ax2.set_xlim('2017-12-15','2018-02-15')
ax2.set_xlim(17471+20,17591-20)


# Migué na legenda
l = ('10', '20', '30')
ax.set_yticks(np.arange(5,34.5,10))
ax.set_yticklabels(['30', '20', '10'])
ax.set_ylabel('Distancia \ndo fundo (m)')
ax.set_ylabel('Distancia \ndo fundo (m)')

plt.savefig('D:/Users/julia/Desktop/Academia/01_Mestrado/Resultados/PerfTemp_2017PF_v6.png',
            format='png', dpi=300, bbox_inches='tight')
plt.show()
